In [2]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
import scipy
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from util import plot_history
import re
import feature_builder

Cargo datasets

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [48]:
kernel_size=1
strides=4

Armo features con el feature_builder

In [63]:
def prepare_for_cnn(df):
    X_train = feature_builder.process_dataset(df).to_numpy()
    X_train = X_train.reshape(X_train.shape[0], kernel_size, X_train.shape[1])
    return X_train

In [49]:
X_train = prepare_for_cnn(train_df)
y_train = train_df['target'].values

Creo el modelo y corro la CNN

In [57]:
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=kernel_size, strides=strides, input_shape=(X_train.shape[1], X_train.shape[2]), activation='relu'))
    model.add(MaxPooling1D(1))
    model.add(Conv1D(filters=128, kernel_size=kernel_size, strides=strides, activation='relu'))
    model.add(MaxPooling1D(1))
    model.add(Conv1D(filters=64, kernel_size=kernel_size, strides=strides, activation='relu'))
    model.add(MaxPooling1D(1))
    model.add(Conv1D(filters=64, kernel_size=kernel_size, strides=strides, activation='relu'))
    model.add(MaxPooling1D(1))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [60]:
model = create_model()
history = model.fit(X_train, y_train, epochs=15, shuffle=True, validation_split=0.0)
plot_history(history)

Epoch 1/15
238/238 [==============================] - 0s 2ms/step - loss: 0.6660 - accuracy: 0.5927
Epoch 2/15
238/238 [==============================] - 0s 2ms/step - loss: 0.5294 - accuracy: 0.7456
Epoch 3/15
238/238 [==============================] - 0s 2ms/step - loss: 0.4890 - accuracy: 0.7743
Epoch 4/15
238/238 [==============================] - 0s 2ms/step - loss: 0.4600 - accuracy: 0.7971
Epoch 5/15
238/238 [==============================] - 0s 2ms/step - loss: 0.4539 - accuracy: 0.8011
Epoch 6/15
238/238 [==============================] - 0s 2ms/step - loss: 0.4415 - accuracy: 0.8090
Epoch 7/15
238/238 [==============================] - 0s 2ms/step - loss: 0.4313 - accuracy: 0.8161
Epoch 8/15
238/238 [==============================] - 0s 2ms/step - loss: 0.4262 - accuracy: 0.8139
Epoch 9/15
238/238 [==============================] - 0s 2ms/step - loss: 0.4273 - accuracy: 0.8130
Epoch 10/15
238/238 [==============================] - 0s 2ms/step - loss: 0.4314 - accuracy: 0.8112

KeyError: 'val_accuracy'

Vemos cuales fallo

In [35]:
prediction = model.predict_classes(X_test)
df = pd.DataFrame({'text': test, 'target': y_test, 'prediction': [x[0] for x in prediction]})

NameError: name 'X_test' is not defined

In [53]:
pd.set_option('display.max_rows', len(df))
pd.set_option('display.max_colwidth', None)
df[df['target'] != df['prediction']]

,text,target,prediction
1,da judge gave dis girl 5pm curfew,0,1
2,us wont upgrade its infrastructure,1,0
4,nigeltanner1 believe it or not we've had too much rain here. our newly planted maple trees are actually drowning.,1,0
9,in response to trauma children of addicts develop a defensive self - one that decreases vulnerability. (3,0,1
13,dad bought a dvd that looks like a science doc on the front but i read the back and it's actually about the impending biblical apocalypse,1,0
15,reddit will now quarantine offensive content,1,0
22,sapphirescallop destroy oppa image oops! there's nothing left right haaaaaa,0,1
23,they should name hurricanes with black people names. i'd be terrified of hurricane shanaynay!,0,1
30,massacre at {1} : has the world forgotten the {1} {1} {1} {1},1,0
31,so when you're caught in a landslidei'll be there for youand in the rain give you sunshinei'll be there for you,0,1


In [54]:
df[df['target'] == df['prediction']]

,text,target,prediction
0,11-year-old boy charged with manslaughter of toddler: report: an 11-year-old boy has been charged with manslaughter over the fatal sh...,1,1
3,dyannbridges yeshayad check out this {1} preview of claytonbryant danger zone coming soon!,0,0
5,no {1} of {1} in {1} {1} {1} {1} {1} {1} {1} {1} {1} {1},1,1
6,potential storm surge flooding map by national hurricane center,1,1
7,olrules welcome - read a free chapter of my new book encounters with jesus. it's full of hope.,0,0
8,on 1st august {1} jets bombed the village zergele in qendil and killed 8 civilians and 15 others wounded,1,1
10,mi_country_hick pfft! i wish i had a bot like that. instead i'm daily inundated with 140 characters of the same unhinged machismo bullshit.,0,0
11,i'm gonna fight taylor as soon as i get there.,0,0
12,deldryden if i press on the twitch will my head explode,0,0
14,i tried making a chocolate and peanut butter lava cake using my {1} protein shake mix and aû_,0,0


Calculemos el test para kaggle

In [65]:
ids = test_df['id']

final_test = prepare_for_cnn(test_df)
prediction = model.predict_classes(final_test)

final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

,target
id,
0,0
2,1
3,1
9,1
11,1
...,...
10861,1
10865,1
10868,1


In [66]:
final_df['target'].value_counts()

0    2135
1    1128
Name: target, dtype: int64

In [67]:
final_df.count()

target    3263
dtype: int64

In [68]:
final_df.to_csv('cnn.csv')